<a href="https://colab.research.google.com/github/AbdulMoiz-MMY/Agentic_AI_Learning/blob/main/project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install -qU langchain-pinecone langchain-google-genai

In [18]:
from google.colab import userdata

from pinecone import Pinecone,ServerlessSpec

pinecone_api_key = userdata.get("PINECONE_API_KEY")

if not pinecone_api_key:
    raise ValueError("PINECONE_API_KEY not set")

pc = Pinecone(api_key = pinecone_api_key)

In [19]:
index_name = "onclass-rag-project"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension= 768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [20]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ['GOOGLE_API_KEY'] = userdata.get("GOOGLE_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [21]:
vector = embeddings.embed_query("We are building a rag query")

In [22]:
vector[:5]

[0.03296078369021416,
 -0.05385661870241165,
 -0.0600518062710762,
 -0.032095782458782196,
 -0.009051025845110416]

In [23]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index,embeddings)

In [24]:
!pip install -q langchain_community

In [25]:
!pip install PyPDF

In [26]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/personal_finance_advisor.pdf")
data = loader.load()

In [27]:
len(data)

22

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

docs = text_splitter.split_documents(data)

len(docs)

66

In [29]:
docs[3]

Document(metadata={'source': '/content/personal_finance_advisor.pdf', 'page': 1}, page_content='How can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Creat

In [30]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(docs))]

vector_store.add_documents(documents=docs,ids=uuids)

['7f765d11-6642-482f-a442-38f9ae10b181',
 '4d5f55c4-e53a-4a65-9cb1-aecb414ae1ef',
 '43e69148-e169-41f1-977a-c24c778f332f',
 '54c9ff24-0fd6-4505-a6aa-05c46e3259a2',
 '2c804601-0900-4bcb-83ac-45aff1f56834',
 '07117a54-c14a-4fbe-ba2d-929016fb4f33',
 '70c29cfc-937a-45b0-81f5-046f5e374d24',
 '43a994a7-8555-4cfa-b85c-e283d12109b9',
 'cee26bcc-4d11-402b-ba53-0d0133f54591',
 '6f3c03ff-b770-471a-9562-646171e41220',
 'd0256ef7-8f53-4a2a-bfab-ca2e60931340',
 '5ac90795-4167-4cfa-b891-2eb34410307f',
 '86e5ebf1-9dbe-4e50-ad90-ffa0c68e13f8',
 '812c9a3f-de60-41fe-be2f-d97ab80bf313',
 '55a586f0-ba14-4f89-a3d6-c3009b8b3be2',
 'f31c9e8c-f6af-4eb8-bea3-617e18a0ccea',
 '16094e26-732a-4eb8-84f5-b1f43be7e659',
 '9f772ad6-1857-423d-9087-c62ff811366c',
 '1115b2f1-6bae-4ac2-b50f-38974e7f5a91',
 'c1c0a1a9-d7d7-4848-bc1c-80c19ac25b13',
 '26e85380-3cc1-4c46-bea5-5257f0bd7635',
 '9a6f6ffb-8769-4d69-a791-b5e1b2b69d38',
 'a297e227-b8eb-481e-812d-01af6f6d4527',
 '98158dbe-781d-4a9d-bdb4-b9a810189f97',
 'd229cc13-bfd5-

In [31]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("Return Policy for Product 452 and 950")

In [32]:
retrieved_docs

[Document(id='ddd9f5e1-6e3b-44db-91c8-73c242fd8296', metadata={'page': 0.0, 'source': '/content/personal_finance_advisor.pdf'}, page_content='Query Advice\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on 

In [33]:
len(retrieved_docs)

10

In [34]:
# print(retrieved_docs[5].page_content)
if retrieved_docs:
    print(retrieved_docs[4].page_content)
else:
    print("retrieved_docs is empty, cannot access element at index 5.")

How can I save money on groceries? Create a budget
How can I save money on rent? Create a budget
How can I save money on utilities? Create a budget
How can I save money on entertainment? Create a budget
How can I save money on travel? Create a budget
How can I save money on groceries? Create a budget
How can I save money on rent? Create a budget
How can I save money on utilities? Create a budget
How can I save money on entertainment? Create a budget
How can I save money on travel? Create a budget
How can I save money on groceries? Create a budget
How can I save money on rent? Create a budget
How can I save money on utilities? Create a budget
How can I save money on entertainment? Create a budget
How can I save money on travel? Create a budget
How can I save money on groceries? Create a budget
How can I save money on rent? Create a budget
How can I save money on utilities? Create a budget
How can I save money on entertainment? Create a budget


In [35]:
results = vector_store.similarity_search(
    "Return Policy for Product 9,50",
    k=10
)

for res in results :
  print(f"* {res.page_content} [{res.metadata}]")

* Query Advice
How can I save money on groceries? Create a budget
How can I save money on rent? Create a budget
How can I save money on utilities? Create a budget
How can I save money on entertainment? Create a budget
How can I save money on travel? Create a budget
How can I save money on groceries? Create a budget
How can I save money on rent? Create a budget
How can I save money on utilities? Create a budget
How can I save money on entertainment? Create a budget
How can I save money on travel? Create a budget
How can I save money on groceries? Create a budget
How can I save money on rent? Create a budget
How can I save money on utilities? Create a budget
How can I save money on entertainment? Create a budget
How can I save money on travel? Create a budget
How can I save money on groceries? Create a budget
How can I save money on rent? Create a budget
How can I save money on utilities? Create a budget
How can I save money on entertainment? Create a budget [{'page': 0.0, 'source': '/co

In [36]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash"
)

In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the followingpieces of retrieved context to answer the question"
    "If you dont know the answer, say that you don't know. Use three sentences maximum and keep answers consise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)



In [38]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain= create_retrieval_chain(retriever,question_answer_chain)

In [39]:
from IPython.display import display,Markdown

response = rag_chain.invoke({"input":"Return Policy for product 625"})
print("Input:", response['input'])
print(f"Context: {response['context'][:2]}")
print("Answer:", response['answer'])

Input: Return Policy for product 625
Context: [Document(id='ddd9f5e1-6e3b-44db-91c8-73c242fd8296', metadata={'page': 0.0, 'source': '/content/personal_finance_advisor.pdf'}, page_content='Query Advice\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on grocer

In [40]:
response = rag_chain.invoke({"input":"Return Policy for product 452 and 950"})
print("Input:", response['input'])
print(f"Context: {response['context'][:2]}")
print("Answer:", response['answer'])

Input: Return Policy for product 452 and 950
Context: [Document(id='ddd9f5e1-6e3b-44db-91c8-73c242fd8296', metadata={'page': 0.0, 'source': '/content/personal_finance_advisor.pdf'}, page_content='Query Advice\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money o

In [41]:
def answer_to_query(query):
    vector_results = vector_store.similarity_search(query, k=10)
    response = llm.invoke(
        f"Answer this query: {query}\nHere are some references: {vector_results}"
    )

    if hasattr(response, 'content'):
        ans = response.content
    else:
        ans = response

    return ans, query, vector_results

In [42]:
ans, query, content = answer_to_query("For Product 652. What is the return policy?")

print(f"Input : {query}")
print(f"Content : {content}")
print(f"Answer : {ans}")

Input : For Product 652. What is the return policy?
Content : [Document(id='ddd9f5e1-6e3b-44db-91c8-73c242fd8296', metadata={'page': 0.0, 'source': '/content/personal_finance_advisor.pdf'}, page_content='Query Advice\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save